In [1]:
#Use this If the dataset has all the data in the format of a single csv file
#Num    abstract	cord_uid
%pip install pandas numpy matplotlib seaborn transformers torch


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [45]:
import pandas as pd
from tqdm import tqdm

In [46]:
def preprocess_data(data_path, sample_size):


  data = pd.read_csv(data_path, low_memory=False)


  data = data.dropna(subset = ['abstract']).reset_index(drop = True)


  data = data.sample(sample_size)[['abstract', 'cord_uid']]

  return data


In [47]:
ls

dataset/             script.ipynb         vector_database.csv
env/                 test.py
model/               tokenizer/


In [48]:
data_path = "/Users/akuldeepj/Desktop/code/LLM_plagerism/dataset/metadata.csv"
source_data = preprocess_data(data_path, 100)

In [49]:
source_data.head()


,abstract,cord_uid
68027,"The lockdown of the country, imposed by the go...",xug40l4o
82707,"During the current COVID-19 pandemic, the rapi...",dck2gn3f
400080,Comparing first and second wave MIS-C cohorts ...,as2cayui
48153,This paper voices the opinions of internationa...,x82982pd
270030,We used a quantitative microbial risk assessme...,7sfmk8eq


In [51]:
%pip install transformers keras torch scikit-learn tensorflow tensorflow-macos tensorflow-metal
import torch
from keras.utils import pad_sequences
from transformers import BertTokenizer,  AutoModelForSequenceClassification
from sklearn.metrics.pairwise import cosine_similarity


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [52]:
model_path = "bert-base-uncased"

tokenizer = BertTokenizer.from_pretrained(model_path, 
                                          do_lower_case=True)

model = AutoModelForSequenceClassification.from_pretrained(model_path,
                                                          output_attentions=False,
                                                          output_hidden_states=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [53]:
ls

dataset/             script.ipynb         vector_database.csv
env/                 test.py
model/               tokenizer/


In [54]:
def create_vector_from_text(tokenizer, model, text, MAX_LEN = 510):
    
    input_ids = tokenizer.encode(
                        text, 
                        add_special_tokens = True, 
                        max_length = MAX_LEN,                           
                   )    

    results = pad_sequences([input_ids], maxlen=MAX_LEN, dtype="long", 
                              truncating="post", padding="post")
    
    
    input_ids = results[0]

    
    attention_mask = [int(i>0) for i in input_ids]
    
    
    input_ids = torch.tensor(input_ids)
    attention_mask = torch.tensor(attention_mask)

    
    input_ids = input_ids.unsqueeze(0)
    attention_mask = attention_mask.unsqueeze(0)
    
    
    model.eval()
    
    
    with torch.no_grad():        
        logits, encoded_layers = model(
                                    input_ids = input_ids, 
                                    token_type_ids = None, 
                                    attention_mask = attention_mask,
                                    return_dict=False)

    layer_i = 12 
    batch_i = 0 
    token_i = 0 
        
    # Embedding.
    vector = encoded_layers[layer_i][batch_i][token_i]


    vector = vector.detach().cpu().numpy()

    return(vector)

In [55]:
import numpy as np

def create_vector_database(data):
    
    
    vectors = []
    
    
    source_data = data.abstract.values
    
    
    for text in tqdm(source_data):
        
    
        vector = create_vector_from_text(tokenizer, model, text)
        
        
        vectors.append(vector)
    
    data["vectors"] = vectors
    data["vectors"] = data["vectors"].apply(lambda emb: np.array(emb))
    data["vectors"] = data["vectors"].apply(lambda emb: emb.reshape(1, -1))
    
    return data

In [56]:
vector_database = create_vector_database(source_data)


100%|██████████| 100/100 [00:15<00:00,  6.41it/s]


In [57]:
vector_database.sample(5)


,abstract,cord_uid,vectors
586612,Porcine hemagglutinating encephalomyelitis vir...,07b3pbxc,"[[-0.3795591, -0.348499, -0.78716296, -0.15551..."
32236,Extrapolating from modern international unders...,fucyt0zc,"[[-0.28831905, -0.41381738, -0.8977126, -0.412..."
200868,COVID-19 is an important threat worldwide. Thi...,msu81brd,"[[-0.6227991, -0.74918747, -0.06620249, -0.415..."
791353,Trauma survivors who suffer from posttraumatic...,nslygsdf,"[[-0.7621305, -0.3840289, -0.671917, -0.228419..."
503719,Understanding the cell entry mechanism of pand...,muzqgk95,"[[-0.28338796, 0.011545289, -0.36038852, -0.50..."


In [58]:
len(vector_database)

100

In [59]:
!pip -q install sentencepiece
from transformers import MarianMTModel, MarianTokenizer


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [60]:
def process_document(text):
   
    text_vect = create_vector_from_text(tokenizer, model, text)
    text_vect = np.array(text_vect)
    text_vect = text_vect.reshape(1, -1)

    return text_vect

    
def is_plagiarism(similarity_score, plagiarism_threshold):

  is_plagiarism = False

  if(similarity_score >= plagiarism_threshold):
    is_plagiarism = True

  return is_plagiarism



def run_plagiarism_analysis(query_text, data, plagiarism_threshold=0.8):

    top_N=3

    query_vect = process_document(query_text)

    data["similarity"] = data["vectors"].apply(lambda x: cosine_similarity(query_vect, x))
    data["similarity"] = data["similarity"].apply(lambda x: x[0][0])

    similar_articles = data.sort_values(by='similarity', ascending=False)[0:top_N+1]
    formated_result = similar_articles[["abstract", "cord_uid", "similarity"]].reset_index(drop = True)

    similarity_score = formated_result.iloc[0]["similarity"] 
    most_similar_article = formated_result.iloc[0]["abstract"] 
    is_plagiarism_bool = is_plagiarism(similarity_score, plagiarism_threshold)

    plagiarism_decision = {'similarity_score': similarity_score, 
                             'is_plagiarism': is_plagiarism_bool,
                             'most_similar_article': most_similar_article, 
                             'article_submitted': query_text
                            }

    return plagiarism_decision
    

In [64]:
new_incoming_text = "My previous experience with flu"

analysis_result = run_plagiarism_analysis(new_incoming_text, vector_database, plagiarism_threshold=0.8)

In [65]:
print(analysis_result)

{'similarity_score': 0.7408559, 'is_plagiarism': False, 'most_similar_article': 'For a searchable version of these abstracts, please visit www.acrabstracts.org.', 'article_submitted': 'My previous experience with flu'}


In [63]:
# Save tokenizer
tokenizer.save_pretrained("tokenizer")

# Save model
model.save_pretrained("model")


In [34]:
# Load tokenizer
loaded_tokenizer = BertTokenizer.from_pretrained("tokenizer")

# Load model
loaded_model = AutoModelForSequenceClassification.from_pretrained("model",
                                                                  output_attentions=False,
                                                                  output_hidden_states=True)
# query_text = "hello I am Akuldeep the plumber and I repair pipes for a living and I am very good at it"
query_text = "Hello I name is ajay and my early life experience has a long lasting experience on social behaviour"
def process_document_with_loaded_model(text):
    text_vect = create_vector_from_text(loaded_tokenizer, loaded_model, text)
    text_vect = np.array(text_vect)
    text_vect = text_vect.reshape(1, -1)
    return text_vect
def run_plagiarism_analysis_with_loaded_model(query_text, data, plagiarism_threshold=0.8):
    top_N = 3
    query_vect = process_document_with_loaded_model(query_text)

    data["similarity"] = data["vectors"].apply(lambda x: cosine_similarity(query_vect, x))
    data["similarity"] = data["similarity"].apply(lambda x: x[0][0])

    similar_articles = data.sort_values(by='similarity', ascending=False)[0:top_N+1]
    formated_result = similar_articles[["abstract", "cord_uid", "similarity"]].reset_index(drop=True)

    similarity_score = formated_result.iloc[0]["similarity"]
    most_similar_article = formated_result.iloc[0]["abstract"]
    is_plagiarism_bool = is_plagiarism(similarity_score, plagiarism_threshold)

    plagiarism_decision = {'similarity_score': similarity_score,
                           'is_plagiarism': is_plagiarism_bool,
                           'most_similar_article': most_similar_article,
                           'article_submitted': query_text
                          }

    print("Plagiarism Analysis Results:")
    print(f"Similarity Score: {similarity_score}")
    print(f"Is Plagiarism: {is_plagiarism_bool}")
    print(f"Most Similar Article:\n{most_similar_article}")
    print(f"Article Submitted:\n{query_text}")

    return plagiarism_decision
plagiarism_result = run_plagiarism_analysis_with_loaded_model(query_text, vector_database)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Plagiarism Analysis Results:
Similarity Score: 0.8608219623565674
Is Plagiarism: True
Most Similar Article:
[ ]there is great potential for attitudes and decision-making processes regarding working while sick to change, hopefully for the better [ ]what once was a largely private and personal decision based on a somewhat limited set of antecedents (see Miraglia & Johns, 2016) is now multifaceted, emotionally charged, and perhaps (hopefully?) more morally derived and other-focused than before Research suggests that emphasizing harm to others, as compared with emphasizing harm to oneself, increases engagement in personal safety behaviors (e g , Grant & Hoffman, 2011) [ ]we suggest additional investigation as to whether considering other stakeholders in decision making can be leveraged for changing presenteeism beliefs and behaviors An opportunity for lasting positive change At present, the many ways in which the COVID-19 pandemic has disrupted the world through premature loss of life, dec

In [25]:
vector_database.to_csv("vector_database.csv", index=False) 